In [1]:
import pandas as pd
import numpy as np
import pickle
import re

In [45]:
df = pd.read_csv('./chart.csv')
df

,HADM_ID,History of Present Illness,Allergies and History,Physical Exam,Major Surgical or Invasive Procedure and Pertinent Result,Brief Hospital Course,DOB,DOA,DOD,drop
0,100001.0,35F w/ poorly controlled Type 1 diabetes melli...,Levaquin <aller>\nType 1 diabetes mellitis w/ ...,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",none <major>\nAdmission Labs: [**2117-9-11**] ...,35 yo F with HTN & poorly controlled type I DM...,35 years old,day 1,day 6,0
1,100003.0,Mr. [**Known lastname 52368**] is a 59M w HepC...,Patient recorded as having No Known Allergies ...,ON ADMISSION\nVS: T95.9 HR 71 BP 83/36 RR 11 ...,EGD\nRight IJ CVL <major>\nLABS ON ADMISSION\n...,Mr [**Known lastname 52368**] is a 59M w HCV c...,59 years old,day 1,day 4,0
2,100007.0,Ms [**Known lastname **] is a 73 year old fema...,Penicillins / Dilantin <aller>\nChronic Pancre...,T: 97.9 HR: 79 BP: 153/60 RR: 22 Spo2 100% on...,Exploratory Laparotomy\nLysis of adhesions\nSm...,Ms [**Known lastname **] was admitted through ...,73 years old,day 1,day 7,0
3,100009.0,60yo man with known coronary disease (AMI in [...,No Known Allergies / Adverse Drug Reactions <a...,Pulse: 58 Resp: 16 O2 sat: 97%-RA\nB/P Rig...,"[**2162-5-17**]: CABGx4 LIMA-> LAD, RSVG-> Dia...",The patient was admitted to the hospital and b...,60 years old,day 1,day 5,0
4,100010.0,"54 y/o female w/Large left renal mass, s/p med...",Penicillins <aller>\nPMH/PSH: Mediastinoscopy...,"NAD, A&Ox3\nNo respiratory distress\nAbd: soft...",Open left radical nephrectomy <major>\n[**2109...,Patient was admitted to Urology after undergoi...,54 years old,day 1,day 4,0
...,...,...,...,...,...,...,...,...,...,...
38063,199986.0,73 yo f with h/o HTN and +fam hx transferred a...,No Drug Allergy Information on File <aller>\ns...,"Vitals: T= 95.9, HR = 61 , BP = 121/69, RR = 2...",cardiac cath <major>\n[**2188-10-30**] 08:40PM...,# CAD: The paitent had an inferior STEMI. Her ...,73 years old,day 1,day 3,0
38064,199987.0,"72 yo male s/p motor vehicle crash, he was hi...",Patient recorded as having No Known Allergies ...,PHYSICAL EXAM\n Temp (F): 98\n Heart R...,[**2175-5-22**] Right femoral percutaneous pla...,He was admitted to the Trauma Service. Orthope...,72 years old,day 1,day 9,0
38065,199992.0,This is an 85 year old lady with a past medica...,Sulfa (Sulfonamide Antibiotics) <aller>\n1. Ob...,Upon admission\nVS: 94.6 66 129/67 13 92% on 2...,None. <major>\nLabs upon admission: Urinalysi...,85 year old lady with a past medical history s...,85 years old,day 1,day 3,0
38066,199998.0,This is a 69 year old male in known coronary a...,Midazolam Hcl <aller>\nCoronary Artery Disease...,"Vitals: 98.2, 122/70, 54, 18, 98%RA\nGeneral: ...",[**2119-2-20**] Off Pump Coronary Artery Bypas...,Mr. [**Known lastname 77739**] was admitted to...,69 years old,day 1,day 6,0


In [39]:
print(df.iloc[4]['History of Present Illness'])

54 y/o female w/Large left renal mass, s/p mediastinoscopy
showing metastatic RCC.  [**Known firstname **] is a previously healthy
patientn who, two years ago, noted one episode of gross
hematuria, which quickly resolved.  More recently, she has had a
50-pound weight loss from 210 to 160 pounds over the past six
months.  She has noted some
fullness in the left upper quadrant, but no actual pain and only
requires very occasional Tylenol for this.  Imaging was
performed, which revealed a very large left renal mass
consistent with renal cell carcinoma.  On [**2109-11-6**], she
underwent a mediastinoscopy with lymph node biopsy, which was
consistent with metastatic carcinoma.  She has noted severe
fatigue and although she can walk at least 10 minutes, her
performance status is lower than normal for her, probably ECOG
1.  No gross hematuria.  No urinary infections or other urinary
symptoms. She has occasional night sweats, no fever, occasional
cough, no hemoptysis. <pres_his>



In [3]:
df1 = pd.read_csv('./mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv')
df1

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."
...,...,...,...,...
14562,14432,V7399,Scrn unspcf viral dis,Special screening examination for unspecified ...
14563,14433,V740,Screening for cholera,Screening examination for cholera
14564,14434,V741,Screening-pulmonary TB,Screening examination for pulmonary tuberculosis
14565,14435,V742,Screening for leprosy,Screening examination for leprosy (Hansen's di...


In [4]:
df2 = pd.read_csv('./mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv')
df2

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254
...,...,...,...,...,...
651042,639798,97503,188195,2.0,20280
651043,639799,97503,188195,3.0,V5869
651044,639800,97503,188195,4.0,V1279
651045,639801,97503,188195,5.0,5275


In [5]:
# concat the long title of ICD code for each patient
ID_list = []
ICD_list = []
for i in range(len(df2)):
    if df2.iloc[i]["HADM_ID"] not in ID_list:
        ID_list.append(df2.iloc[i]["HADM_ID"])
        code = df1.loc[df1["ICD9_CODE"] == df2.iloc[i]["ICD9_CODE"]]
        if not code.empty:
            ICD_list.append(code.iloc[0]["LONG_TITLE"])
        else:
            ICD_list.append('')
    else:
        idx = ID_list.index(df2.iloc[i]["HADM_ID"])
        code = df1.loc[df1["ICD9_CODE"] == df2.iloc[i]["ICD9_CODE"]]
        if not code.empty:
            ICD_list[idx] += ' ' + code.iloc[0]["LONG_TITLE"]

In [29]:
ICD_dict = {"ID":ID_list, "ICD":ICD_list}
with open('ICD.pk', 'wb') as f:
    pickle.dump(ICD_dict, f)

In [ ]:
# vectorize the ICD code
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ICD_list)
with open('vectorizer.pk', 'wb') as f:
    pickle.dump(X, f)
print(X.shape)

(58976, 3847)


In [14]:
with open('vectorizer.pk', 'rb') as f:
    X = pickle.load(f)

In [23]:
# calculate cosine similarity between each pair of patients
from sklearn.metrics.pairwise import cosine_similarity
result = []
for i in range(int(X.shape[0]/100)+1):
    if (i+1)*100 > X.shape[0]:
      tmp = cosine_similarity(X[i*100:], X)
    else:
      tmp = cosine_similarity(X[i*100:(i+1)*100], X)
    tmp[np.isclose(tmp, 1)] = 0
    result += list(np.argmax(tmp, axis=1))


In [30]:
with open('ICD.pk', 'rb') as f:
    ICD_dict = pickle.load(f)
ID2_list = []
for i in range(len(result)):
    ID2_list.append(ICD_dict["ID"][result[i]])
ICD_dict['ID2'] = ID2_list
with open('ICD.pk', 'wb') as f:
    pickle.dump(ICD_dict, f)

In [ ]:
df = df.drop(columns=['DOB', 'DOA', 'DOD', 'drop'])

In [61]:
# create negative pairs of nli dataset
df3 = df.copy()
result = []
for i in range(len(df)):
    idx = ICD_dict['ID'].index(df3.iloc[i]['HADM_ID'])
    tmp = df.loc[df['HADM_ID'] == ICD_dict['ID2'][idx]]
    if not tmp.empty:
        result.append(tmp.iloc[0]['Brief Hospital Course'])
    else:
        result.append('')
df['label'] = [1 for i in range(len(df))]
df3['label'] = [0 for i in range(len(df))]
df3['Brief Hospital Course'] = result

In [64]:
# remove rows with empty data
df3 = df3.drop(df3[df3['Brief Hospital Course'] == ''].index)

In [68]:
df4 = pd.concat([df, df3])
df4 = df4.drop(columns=['HADM_ID'])
df4

,History of Present Illness,Allergies and History,Physical Exam,Major Surgical or Invasive Procedure and Pertinent Result,Brief Hospital Course,label
0,35F w/ poorly controlled Type 1 diabetes melli...,Levaquin <aller>\nType 1 diabetes mellitis w/ ...,"GEN: Awake, alert, and oriented\nHEENT: PERRLA...",none <major>\nAdmission Labs: [**2117-9-11**] ...,35 yo F with HTN & poorly controlled type I DM...,1
1,Mr. [**Known lastname 52368**] is a 59M w HepC...,Patient recorded as having No Known Allergies ...,ON ADMISSION\nVS: T95.9 HR 71 BP 83/36 RR 11 ...,EGD\nRight IJ CVL <major>\nLABS ON ADMISSION\n...,Mr [**Known lastname 52368**] is a 59M w HCV c...,1
2,Ms [**Known lastname **] is a 73 year old fema...,Penicillins / Dilantin <aller>\nChronic Pancre...,T: 97.9 HR: 79 BP: 153/60 RR: 22 Spo2 100% on...,Exploratory Laparotomy\nLysis of adhesions\nSm...,Ms [**Known lastname **] was admitted through ...,1
3,60yo man with known coronary disease (AMI in [...,No Known Allergies / Adverse Drug Reactions <a...,Pulse: 58 Resp: 16 O2 sat: 97%-RA\nB/P Rig...,"[**2162-5-17**]: CABGx4 LIMA-> LAD, RSVG-> Dia...",The patient was admitted to the hospital and b...,1
4,"54 y/o female w/Large left renal mass, s/p med...",Penicillins <aller>\nPMH/PSH: Mediastinoscopy...,"NAD, A&Ox3\nNo respiratory distress\nAbd: soft...",Open left radical nephrectomy <major>\n[**2109...,Patient was admitted to Urology after undergoi...,1
...,...,...,...,...,...,...
38062,Mrs. [**Known lastname 15308**] is a 71 year o...,Codeine / Diltiazem / Doxycycline / Hydrocodon...,VS: T= 98.4 BP= 114/65 HR= 77 RR= 23 O2 sat= 9...,Cardiac Catheterization <major>\nECHO [**2191-...,80 y/o M with history of HTN and GERD presente...,0
38063,73 yo f with h/o HTN and +fam hx transferred a...,No Drug Allergy Information on File <aller>\ns...,"Vitals: T= 95.9, HR = 61 , BP = 121/69, RR = 2...",cardiac cath <major>\n[**2188-10-30**] 08:40PM...,"60F with HTN, dyslipidemia, and family history...",0
38065,This is an 85 year old lady with a past medica...,Sulfa (Sulfonamide Antibiotics) <aller>\n1. Ob...,Upon admission\nVS: 94.6 66 129/67 13 92% on 2...,None. <major>\nLabs upon admission: Urinalysi...,ICU course\n# Hypothermia: The patient had a ...,0
38066,This is a 69 year old male in known coronary a...,Midazolam Hcl <aller>\nCoronary Artery Disease...,"Vitals: 98.2, 122/70, 54, 18, 98%RA\nGeneral: ...",[**2119-2-20**] Off Pump Coronary Artery Bypas...,Admitted [**7-17**] and underwent CABG x 3 wit...,0


In [69]:
df4.to_csv('nli_dataset.csv', index=False)